### Packages

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from utilsforecast.preprocessing import fill_gaps
import lightgbm as lgb
import xgboost as xgb
from mlforecast import MLForecast
from utilsforecast.losses import mape
from sklearn.metrics import mean_absolute_percentage_error

### Load Data

In [22]:
df = pd.read_csv("../0-Data/train_0irEZ2H.csv")
df

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20
1,2,17/01/11,8091,216419,99.0375,99.0375,0,0,28
2,3,17/01/11,8091,216425,133.9500,133.9500,0,0,19
3,4,17/01/11,8091,216233,133.9500,133.9500,0,0,44
4,5,17/01/11,8091,217390,141.0750,141.0750,0,0,52
...,...,...,...,...,...,...,...,...,...
150145,212638,09/07/13,9984,223245,235.8375,235.8375,0,0,38
150146,212639,09/07/13,9984,223153,235.8375,235.8375,0,0,30
150147,212642,09/07/13,9984,245338,357.6750,483.7875,1,1,31
150148,212643,09/07/13,9984,547934,141.7875,191.6625,0,1,12


In [23]:
df["week"] = pd.to_datetime(df["week"], format="%d/%m/%y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150150 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   record_ID        150150 non-null  int64         
 1   week             150150 non-null  datetime64[ns]
 2   store_id         150150 non-null  int64         
 3   sku_id           150150 non-null  int64         
 4   total_price      150149 non-null  float64       
 5   base_price       150150 non-null  float64       
 6   is_featured_sku  150150 non-null  int64         
 7   is_display_sku   150150 non-null  int64         
 8   units_sold       150150 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 10.3 MB


In [24]:
df

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,2011-01-17,8091,216418,99.0375,111.8625,0,0,20
1,2,2011-01-17,8091,216419,99.0375,99.0375,0,0,28
2,3,2011-01-17,8091,216425,133.9500,133.9500,0,0,19
3,4,2011-01-17,8091,216233,133.9500,133.9500,0,0,44
4,5,2011-01-17,8091,217390,141.0750,141.0750,0,0,52
...,...,...,...,...,...,...,...,...,...
150145,212638,2013-07-09,9984,223245,235.8375,235.8375,0,0,38
150146,212639,2013-07-09,9984,223153,235.8375,235.8375,0,0,30
150147,212642,2013-07-09,9984,245338,357.6750,483.7875,1,1,31
150148,212643,2013-07-09,9984,547934,141.7875,191.6625,0,1,12


### Data Aggregations

In [25]:
store_sales = df.groupby(["store_id", "week"])["units_sold"].sum()
store_sales

store_id  week      
8023      2011-01-17    2344
          2011-01-24    2034
          2011-01-31    2058
          2011-02-07    2152
          2011-02-14    3512
                        ... 
9984      2013-06-11     575
          2013-06-18     622
          2013-06-25     796
          2013-07-02     677
          2013-07-09     662
Name: units_sold, Length: 9880, dtype: int64

In [26]:
store_sales.index

MultiIndex([(8023, '2011-01-17'),
            (8023, '2011-01-24'),
            (8023, '2011-01-31'),
            (8023, '2011-02-07'),
            (8023, '2011-02-14'),
            (8023, '2011-02-21'),
            (8023, '2011-02-28'),
            (8023, '2011-03-07'),
            (8023, '2011-03-14'),
            (8023, '2011-03-21'),
            ...
            (9984, '2013-05-07'),
            (9984, '2013-05-14'),
            (9984, '2013-05-21'),
            (9984, '2013-05-28'),
            (9984, '2013-06-04'),
            (9984, '2013-06-11'),
            (9984, '2013-06-18'),
            (9984, '2013-06-25'),
            (9984, '2013-07-02'),
            (9984, '2013-07-09')],
           names=['store_id', 'week'], length=9880)

In [28]:
store_sales = pd.DataFrame(
    {
        "unique_id": [i[0] for i in store_sales.index],
        "week": [i[1] for i in store_sales.index],
        "units_sold": store_sales.values
    }
)
store_sales

,unique_id,week,units_sold
0,8023,2011-01-17,2344
1,8023,2011-01-24,2034
2,8023,2011-01-31,2058
3,8023,2011-02-07,2152
4,8023,2011-02-14,3512
...,...,...,...
9875,9984,2013-06-11,575
9876,9984,2013-06-18,622
9877,9984,2013-06-25,796
9878,9984,2013-07-02,677


In [29]:
store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9880 entries, 0 to 9879
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   unique_id   9880 non-null   int64         
 1   week        9880 non-null   datetime64[ns]
 2   units_sold  9880 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 231.7 KB
